In [0]:
import numpy as np
np.random.seed(1001)
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
#import tensorflow as tf
import pandas as pd
from keras.preprocessing.sequence import pad_sequences
import pickle

Instructions for updating:
non-resource variables are not supported in the long term


Using TensorFlow backend.


In [0]:
import data_utils
import glove_utils
import display_utils

In [0]:
import models

In [0]:
VOCAB_SIZE  = 50000
with open('dataset_%d.pkl' %VOCAB_SIZE, 'rb') as f:
    dataset = pickle.load(f)
    
doc_len = [len(dataset.test_seqs2[i]) for i in 
           range(len(dataset.test_seqs2))]

In [0]:
dist_mat = np.load('dist_counter_%d.npy' %VOCAB_SIZE)
# Prevent returning 0 as most similar word because it is not part of the dictionary
dist_mat[0,:] = 100000
dist_mat[:,0] = 100000

skip_list = np.load('missed_embeddings_counter_%d.npy' %VOCAB_SIZE)

In [0]:
max_len = 250
train_x = pad_sequences(dataset.train_seqs2, maxlen=max_len, padding='post')
train_y = np.array(dataset.train_y)
test_x = pad_sequences(dataset.test_seqs2, maxlen=max_len, padding='post')
test_y = np.array(dataset.test_y)

In [0]:
tf.reset_default_graph()
if tf.get_default_session():
    sess.close()
sess = tf.Session()
batch_size = 1
lstm_size = 128
#max_len =  100

with tf.variable_scope('imdb', reuse=False):
    model = models.SentimentModel(batch_size=batch_size,
                           lstm_size = lstm_size,
                           max_len = max_len,
                           embeddings_dim=300, vocab_size=dist_mat.shape[1],is_train = False)
saver = tf.train.Saver()
saver.restore(sess, 'imdb_model')

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

INFO:tensorflow:Restoring parameters from imdb_model


In [0]:
visual_idx = np.random.choice(25000)
i=24997
display_utils.visualize_attack(sess, model, dataset, test_x[i], test_x[i])

Original Prediction = Negative. (Confidence = 100.00) 


---------  After attack -------------
New Prediction = Negative. (Confidence = 100.00) 


In [0]:
df = pd.read_csv('reviewwordfinal.csv')
lab = df['Label']
typ = df['type']
df.shape

(25000, 7)

In [0]:
i=12497
print(lab[i])
print(lab.size)
print(df.Review[i])

neg
25000
The basic genre is a thriller intercut with an uncomfortable menage-a-trois. Fellowes has tried to make a lot more out of this, using the lies of the title in order to bring about all manner of small twists, invariably designed to surprise the characters more than the audience.<br /><br />It's really rather messy though. Fellowes doesn't seem interested presenting the thriller elements in a fashion that will keep us seat-edged. Rather his focus is on the moral predicaments themselves.<br /><br />The dialogue is inconsistent, stagey here, vernacular there and with the constant surprise of realism undone by the occasional cliché-landmine. Though there is no fussing over the locations so that the actors can get on with existing in their space the dreadful score can't create a further dimension and often works against the emotional momentum of given set pieces. There's also a very prosaic, dare I say it British feel to the filming. I didn't want to see a document of two successfu

In [0]:
for i in range(0,lab.size):
    if i<12500:    #negative
        x_orig = test_x[i+12500]
        orig_label = test_y[i+12500]
        orig_preds =  model.predict(sess, x_orig[np.newaxis, :])[0]
        if np.argmax(orig_preds) == 1:
            df.loc[i,'orig_sentiment'] = 'pos'
        else:
            df.loc[i,'orig_sentiment'] = 'neg'
        
        df.loc[i,'orig_confidence'] = round(np.max(orig_preds),4)
        
    else:         #positive
        x_orig = test_x[i-12500]
        orig_label = test_y[i-12500]
        orig_preds =  model.predict(sess, x_orig[np.newaxis, :])[0]
        if np.argmax(orig_preds) == 1:
            df.loc[i,'orig_sentiment'] = 'pos'
        else:
            df.loc[i,'orig_sentiment'] = 'neg'
        
        df.loc[i,'orig_confidence'] = round(np.max(orig_preds),4)

In [0]:
df.to_pickle('origpred.pkl')

In [0]:
df.to_csv('originalpred.csv')

In [0]:
with open('original_pred.pkl', 'wb') as f:
    pickle.dump(df, f)